In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import rff

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [17]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [18]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [20]:
model = NeuralNetwork()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299634  [   64/60000]
loss: 2.299810  [ 6464/60000]
loss: 2.276370  [12864/60000]
loss: 2.278308  [19264/60000]
loss: 2.247635  [25664/60000]
loss: 2.224812  [32064/60000]
loss: 2.233437  [38464/60000]
loss: 2.195992  [44864/60000]
loss: 2.185740  [51264/60000]
loss: 2.164205  [57664/60000]
Test Error: 
 Accuracy: 44.1%, Avg loss: 2.160504 

Epoch 2
-------------------------------
loss: 2.165354  [   64/60000]
loss: 2.165559  [ 6464/60000]
loss: 2.099150  [12864/60000]
loss: 2.123393  [19264/60000]
loss: 2.057441  [25664/60000]
loss: 2.006664  [32064/60000]
loss: 2.037393  [38464/60000]
loss: 1.950215  [44864/60000]
loss: 1.947223  [51264/60000]
loss: 1.888301  [57664/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.886476 

Epoch 3
-------------------------------
loss: 1.911744  [   64/60000]
loss: 1.894083  [ 6464/60000]
loss: 1.761536  [12864/60000]
loss: 1.817104  [19264/60000]
loss: 1.689472  [25664/60000]
loss: 1.652105  [32064/600

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [62]:
def sample_xi(nin, nout, sigma=1):
    return torch.randn((nout, nin)) * sigma

@torch.jit.script
def RandomFourierFeatureMap(xi: torch.Tensor, x: torch.Tensor):
    assert xi.shape[1] == x.shape[0]
    return torch.cos(torch.matmul(xi, x))

class RandomFourierFeatureLayer(nn.Module):
    def __init__(self, nin, nout, sigma=1) -> None:
        super().__init__()
        self.register_buffer("xi", sample_xi(nin, nout, sigma))

    def forward(self, x):
        return RandomFourierFeatureMap(self.xi, x)



In [70]:
xi = sample_xi(3, 10, sigma=1)
x = torch.tensor([1,2,3], dtype=torch.float32)
RandomFourierFeatureMap(xi, x)
t = RandomFourierFeatureLayer(3, 10, sigma=0.7)

In [75]:

t.forward(x)

tensor([-0.9377,  0.9937, -0.0632, -0.0609,  0.8517,  0.4634, -0.4303, -0.9407,
        -0.8976,  0.9990])